In [35]:
import os
import fitz
import spacy
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

import argparse

pd.set_option('display.max_colwidth', None)


In [36]:
input_path = os.path.join(os.getcwd(), '..', 'src', 'data')
input_path = os.path.normpath(input_path)

print(f"Input path: {input_path}")

Input path: /Users/lili/Projects_studia/_PORTFOLIO/REMOTE/nlp-group2/src/data


# utils_dataprep.py - bez zmian

Testing data preparation functions

In [37]:
#branch 12
import os
import fitz

def read_documents(input_path):
    """
    Read text content from the first four pages of each PDF document in the specified folder.

    Args:
    - input_path (str): The path to the folder containing the PDF documents.

    Returns:
    - documents (list): A list of lists, where each inner list contains the filename
      and "CHCPL" - the combined text content of the first four pages of the corresponding PDF document.
    """
    documents = []
    if os.path.isdir(input_path):
        for filename in sorted(os.listdir(input_path)):  # Sort to ensure consistent order
            file_path = os.path.join(input_path, filename)
            if filename.endswith('.pdf'):
                try:
                    with fitz.open(file_path) as pdf:
                        combined_text = ""
                        for page_number in range(min(4, pdf.page_count)):
                            try:
                                page = pdf.load_page(page_number)
                                footer = 80  
                                page_text = page.get_text("text", clip=(0, 0, page.rect.width, page.rect.height - footer))
                                combined_text += page_text + " "
                            except Exception as e:
                                pass 
                        combined_text = combined_text.replace("\n", "")
                        documents.append([filename, combined_text])
                except Exception as e:
                    pass 
    else:
        print("Invalid input path. Please provide a valid folder path.")
    return documents


In [38]:
def load_to_pd(input_path):
    """
    Load text data from PDF documents in the specified folder into a pandas DataFrame.

    Args:
    - input_folder (str): The path to the folder containing the PDF documents.

    Returns:
    - text_df (DataFrame): A pandas DataFrame containing the text data from the PDF documents.
      The DataFrame has two columns: "filename" and "CHPCL". Each row corresponds to a PDF document,
      where "filename" contains the name of the PDF file, and "CHPCL" contains the combined text content
      of the first four pages of the PDF.

    """
    text = read_documents(input_path)
    text_df = pd.DataFrame(text)
    current_columns = text_df.columns
    new_columns = [f"column{i+1}" for i in range(len(current_columns))]
    text_df.columns = new_columns
    text_df.rename(columns={"column1": "filename", "column2":"CHPCL"}, inplace=True)
    return text_df

In [39]:
text_df = load_to_pd(input_path)

In [40]:
text_df.head()

filename  \
0          Charakterystyka-10025-2021-12-22-8574_N-2022-01-05.pdf   
1         Charakterystyka-10062-2023-01-11-11269_A-2023-01-22.pdf   
2  Charakterystyka-10088-20200721000000-6861_N-20200916000402.pdf   
3         Charakterystyka-10174-2023-04-12-13484_A-2023-04-29.pdf   
4         Charakterystyka-10252-2022-06-17-10632_A-2022-06-24.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [41]:
text_df.iloc[865:866 ,:]

filename  \
865  Charakterystyka-47283-2023-10-31-15084_D-2023-11-10.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# utils_info_extract.py

Testing functions extracting information from the loaded text corpus.

In [42]:
import fitz
import re
import spacy
import os
import pandas as pd

def apply_replacements(df, column):
    """
    Apply replacements to the specified column in the DataFrame.

    Args:
    - df (DataFrame): The DataFrame.
    - column (str): The name of the column to apply replacements to.
    - replacements (dict): A dictionary where keys are old values to be replaced
      and values are new values.

    Returns:
    - df (DataFrame): The DataFrame with replacements applied to the specified column.
    """
    replacements = {
        "NAZWA WŁASNA": "NAZWA",
        "LECZNICZNEGO": "LECZNICZEGO ",
        "LECZNICZEGO": "LECZNICZEGO ",
        "PODUKTU": "PRODUKTU",
        "PRODUKU": "PRODUKTU",
        "PODUKTU": "PRODUKTU",
        "1.NAZWA ": "1. NAZWA ",
        "1.NAZWA ": "1. NAZWA ",
        "1 nazwa" :"1. NAZWA ",
        "SKLAD": "SKŁAD",
        "2. skład" : "2. SKŁAD ",
        "2.SKŁAD ": "2. SKŁAD ",
        "2 SKŁAD" :"2. SKŁAD ",
        "2 skład": "2. SKŁAD ",
        "4.1 Wskazania": "4.1. Wskazania",
        "4.1Wskazania": "4.1. Wskazania",
        "4.1.Wskazania": "4.1. Wskazania",
        "2.0\\4-": "",
        "\\":"",
        "/":"",
        "jest wskazany": "wskazania"        
    }
    for old_value, new_value in replacements.items():
        df[column] = df[column].str.upper().str.replace(old_value, new_value).str.lower()
    return df


def extract_columns(df, column):
    df = apply_replacements(df, column) 
    for index, row in df.iterrows():
        #nazwa
        start_index_nazwa = row[column].find("1")
        end_index_nazwa = row[column].find("2.") if row[column].find("2.") != -1 else row[column].find("2")
        nazwa = row[column][start_index_nazwa:end_index_nazwa].strip()
        df.at[index, 'nazwa'] = nazwa    
        #sklad
        start_index_sklad = row[column].find("2.") if row[column].find("2.") != -1 else row[column].find("2 skład ilościowy i jakościowy ")
        end_index_sklad = row[column].find("3.") if row[column].find("3.") != -1 else row[column].find("4.1")
        sklad = row[column][start_index_sklad:end_index_sklad].strip()
        df.at[index, 'sklad'] = sklad
        #wskazania
        start_index_wskazania = row[column].find("4.1") if row[column].find("4.1") != -1 else row[column].find("wskazania") 
        end_index_wskazania = row[column].find("4.2")
        wskazania = row[column][start_index_wskazania:end_index_wskazania].strip()
        df.at[index, 'wskazania'] = wskazania
        
    return df[['filename', 'nazwa', 'sklad', 'wskazania']]


In [43]:
text_extracted = extract_columns(text_df, 'CHPCL') 

In [44]:
text_extracted.head()

,filename,nazwa,sklad,wskazania
0,Charakterystyka-10025-2021-12-22-8574_N-2022-01-05.pdf,"1. nazwa produktu leczniczego metazydyna, 20 mg, tabletki powlekane","2. skład jakościowy i ilościowy 1 tabletka zawiera 20 mg trimetazydyny dichlorowodorku (trimetazidini dihydrochloridum). substancje pomocnicze o znanym działaniu: 0,05 mg czerwieni koszenilowej. pełny wykaz substancji pomocniczych, patrz: punkt 6.1.","4.1 wskazania do stosowania trimetazydyna jest wskazana u dorosłych w terapii skojarzonej w objawowym leczeniu pacjentów ze stabilną dławicą piersiową, u których leczenie przeciwdławicowe pierwszego rzutu jest niewystarczające lub nie jest tolerowane."
1,Charakterystyka-10062-2023-01-11-11269_A-2023-01-22.pdf,"1. nazwa produktu leczniczego galpent, 100 mg, tabletki","2. skład jakościowy i ilościowy jedna tabletka zawiera 100 mg pentaerytrytylu tetraazotanu (pentaerythrityli tetranitras) w postaci 33,3% rozcierki laktozowej. substancje pomocnicze o znanym działaniu: sacharoza 3,5 mg i laktoza jednowodna 214,7 mg. pełny wykaz substancji pomocniczych, patrz punkt 6.1.",4.1 wskazania do stosowania stabilna choroba niedokrwienna serca.
2,Charakterystyka-10088-20200721000000-6861_N-20200916000402.pdf,"1. nazwa produktu leczniczego teveten, 600 mg, tabletki powlekane","2. skład jakościowy i ilościowy każda tabletka powlekana zawiera 600 mg eprosartanu (eprosartanum) w postaci eprosartanu mezylanu. substancja pomocnicza o znanym działaniu: każda tabletka powlekana zawiera 43,3 mg laktozy jednowodnej. pełny wykaz substancji pomocniczych, patrz punkt 6.1.",4.1 wskazania do stosowania eprosartan jest wskazany w leczeniu nadciśnienia tętniczego pierwotnego.
3,Charakterystyka-10174-2023-04-12-13484_A-2023-04-29.pdf,"1. nazwa produktu leczniczego 0,9% sodium chloride–braun, 9 mgml, roztwór do infuzji","2. skład jakościowy i ilościowy 1 ml roztworu zawiera sodu chlorek 9,0 mg stężenia elektrolitów: sód 154 mmoll chlorek 154 mmoll pełny wykaz substancji pomocniczych, patrz punkt 6.1.",4.1. wskazania do stosowania • uzupełnianie płynów i elektrolitów w alkalozie hipochloremicznej • niedobór chlorków • krótkoterminowe uzupełnianie objętości śródnaczyniowej • odwodnienie hipotoniczne lub izotoniczne • roztwór do rozpuszczania stężonych elektrolitów i leków nie wykazujących niezgodności farmaceutycznych • zewnętrznie do płukania ran oraz nawilżania opatrunków i tamponów na ranach
4,Charakterystyka-10252-2022-06-17-10632_A-2022-06-24.pdf,"1. nazwa produktu leczniczego cerutin, 100 mg + 25 mg, tabletki powlekane","2. skład jakościowy i ilościowy jedna tabletka powlekana zawiera 100 mg kwasu askorbowego (acidum ascorbicum) i 25 mg rutozydu trójwodnego (rutosidum trihydricum). substancja pomocnicza o znanym działaniu: laktoza jednowodna, 1,90 mg w tabletce pełny wykaz substancji pomocniczych, patrz punkt 6.1.","4.1 wskazania do stosowania − w stanach niedoboru witaminy c i rutozydu – wspomaganie odporności, − uszczelnienie naczyń włosowatych – zapobieganie przeziębieniom i łagodzenie objawów grypy."


In [45]:
import spacy
import re
import pandas as pd
from datetime import datetime
import os

nlp = spacy.load("pl_core_news_lg")


def clean_formatting(text):
    """
    Clean and preprocess the input text by removing punctuation, converting to lowercase, 
    replacing multiple spaces with a single space, and removing digits.
    
    Args:
    - text (str): The input text to be cleaned.
    
    Returns:
    - str: The cleaned and preprocessed text.
    """
    text = re.sub(r'[^\w\s]', '', text)  
    text = text.replace('\n', ' ').strip().lower()  
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'\d+', '', text)  
    return text

def remove_stop_words(tokens, specified_words_to_remove):
    """
    Remove specified stop words from the input text.
    
    Args:
    - text (str): The input text from which stop words will be removed.
    - specified_words_to_remove (list): A list of stop words to be removed from the text.
    
    Returns:
    - str: The input text with specified stop words removed.
    """
    filtered_tokens = [token for token in tokens if token not in specified_words_to_remove]
    return ' '.join(filtered_tokens)

def remove_duplicated_words(text):
    """
    Remove duplicate words from the input text while maintaining the original order.
    
    Args:
    - text (str): The input text from which duplicate words will be removed.
    
    Returns:
    - str: The input text with duplicate words removed.
    """
    words = text.split()
    unique_words = list(dict.fromkeys(words))  # Maintain order and remove duplicates
    return ' '.join(unique_words)

def tokenize_text(text):
    """
    Tokenize the input text using spaCy.

    Args:
    - text (str): The input text to be tokenized.

    Returns:
    - list: A list of tokenized words.
    """
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

def lemmatize_text(text):
    """
    Lemmatize the input text using a specified spaCy language model.
    
    Args:
    - text (str): The input text to be lemmatized.
    
    Returns:
    - str: The lemmatized text.
    """
    doc = nlp(text)  
    lemmatized_text = " ".join(token.lemma_ for token in doc if token.pos_ == "NOUN" )  
    return lemmatized_text


def process_text_columns(df):
    """
    Preprocess text data in specified DataFrame columns.

    Args:
    - df (DataFrame): The input DataFrame containing text columns to be processed.

    Returns:
    - DataFrame: The DataFrame with processed text columns.

    The function iterates over specified columns ('nazwa', 'sklad', 'wskazania') in the DataFrame and performs the following preprocessing steps:
    - For the 'nazwa' column:
        - Clean the text formatting.
        - Remove specified stop words.
        - Remove duplicated words.
    - For the 'sklad' column:
        - Clean the text formatting.
        - Remove specified stop words.
        - Lemmatize the text.
        - Remove duplicated words.
    - For the 'wskazania' column:
        - Clean the text formatting.
        - Remove specified stop words.
        - Lemmatize the text.
        - Remove duplicated words.

    The stop words and text cleaning operations are customized for each column to ensure appropriate preprocessing based on the nature of the text data in each column.
    """
    columns_to_process = ['nazwa', 'sklad', 'wskazania']
    stop_words_nazwa = ['nazwa', 'produktu', 'leczniczego', 'mg', 'ithib', 'charakterystyka', 'roztwór', 'aerozol', 'tabletka', 'kapsułka', 'tabletki', 'inhalacyjny', 'lek', 'wstrzykiwań', 'powlekana',\
                'powlekany', 'powlekane', 'kapsułki', 'twarde', 'twarda', 'ampułkostrzykawce','ampułkostrzykawka', 'infuzji', 'żel', 'żucia', "proszek", 'sporządzania','mgg', 'koncentrat', "mgml", "ml", "jm", \
                 "summary", "of" , "product", "characteristics", "rozpuszczalnik", "roztworu", "dojelitowe", "przedłużonym", "uwalnianiu", "krople", "nos", "nosa", "dawka", "dawkę", \
                    "zawiesina", "zawiesinę", "mikrogramy", "donosową", "maść"]
    stop_words_sklad = ['skład', 'jakościowy', 'ilościowy',  "pełny", "wykaz",  "substancji", "twarda", "twarda", "powlekana",\
                        "pomocniczych", "patrz", "punkt", "substancje","substancja", "pomocnicze"," pomocnicza", "mg", "znanym", "każdy", "każda", \
                        "chlorowodorek", "chlorowodorku", "laktoza jednowodna", "substancja", "pomocnicza" , "kapsułka", "kapsułki",\
                        "tabletka", "tabletce" "powlekana", "tabletki", "powlekane", "sodowy", "sodowego", "otoczka", "otoczką", \
                        "laktozy", "laktoza", "jednowodna", "jednowodnej", "laktoza", "jednowodna", "czerwień", "koszelinowa", "czerwieni", "koszenilowej", "fiolka", "fiolki", \
                        "woda", "wody", "postaci", "sacharoza", "patrz", "punkt","stężenia", "elektrolitów",\
                        "znanym" ,"działaniu", "zawiera", "działanie", "działaniu", "oraz", "lub", "gram", "około", "mililitr", "zawiesina","zawiesiny", \
                        "mmoll", "roztwór", "roztworu",  "wyciąg", "zawiera", "pełny", "wykaz substancji pomocniczych", "do wstrzykiwań", "suchego standaryzowanego", "co najmniej", \
                            "dla produktu",\
                        "w ilości odpowiadającej", "szybko uwalnianej", "wolno uwalnianej", "ampułko-strzykawka", "rozgryzania", "żucia" , "dawka", "dawce", "dawkach"\
                        "odmierzona", "mikrogramów", "ustnik", "zawiesiny", "sorbitol","mgml", "aspartam", "zmikronizowanej", "przeliczeniu", \
                        "alkohol cetostearylowy", "kwas sorbowy", "ampułka", "j.m.", "ml", "aktywności", "wody", "otrzymywaną", "wyniku", "tabletka dojelitowa", "elastyczna", \
                        "lecytyna sojowa", "sorbitol", "sorbitolu",\
                        "postać farmaceutyczna", "wymiary", "żelatynowej", "meql", "infuzji", "przypadku", "zasobnik", "zasobnika", "ilość", "inhalator", "inhalatora"]
    stop_words_wskazania = [ "wskazania", "wskazany" ,  "stosowania","stosowanie", "zapobieganie",  "wskazanie","wskazanym", "wskazane", "wskazana", "wskazanego", "wskazanej", "wskazanym", "wskazanych", \
                            "wskazanymi", "wskazanemu", "wskazaną", "wskazani", "leczniczy", "leczenie", "leczeniu","leczenia", " oraz ", " kiedy ", \
                             " który ", " która ", " które ", " których ",  "pacjent", "patrz", "punkty", "osób", "produkt", "substancja", "pomocniczy", "pomocnicza" , \
                            "metody", "celu", "mieszaną", "obejmującego", "stosowanie", "pacjent", "pacjentom", "pacjentowi", "pacjenta", "w celu", "jest", "są", "jako", \
                            "terapia", "dodatkowy", "dodatkowa", "dodatkowe", "standardowe", \
                            "wstrzykiwań", "suchego standaryzowanego", "co najmniej", "dla produktu",\
                        "w ilości odpowiadającej", "szybko uwalnianej", "wolno uwalnianej", "ampułko-strzykawka", "rozgryzania", "żucia" , "dawka", "dawce", "dawkach"\
                        "odmierzona", "mikrogramów", "ustnik", "zawiesiny", "sorbitol","glikol propylenowy","mgml", "benzoesan sodu", "aspartam", "zmikronizowanej", "w przeliczeniu",
                        "alkohol cetostearylowy", "kwas sorbowy", "ampułka", "j.m.", "ml", "aktywności", "wody", "otrzymywaną", "wyniku", "tabletka dojelitowa", "elastyczna", "lecytyna sojowa", \
                        "postać farmaceutyczna", "wymiary", "żelatynowej", "meql", "infuzji", "postaci", "przypadku", "zasobnik", "zasobnika", "ilość", "inhalator", "inhalatora"]
    
    for column in columns_to_process:
        if column == 'nazwa':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(tokenize_text)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_nazwa)
           
            df.loc[:, column] = df[column].apply(remove_duplicated_words)


        elif column == 'sklad':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(tokenize_text)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_sklad)
            df.loc[:, column] = df[column].apply(lemmatize_text)
            df.loc[:, column] = df[column].apply(remove_duplicated_words)
            
        
        elif column == 'wskazania':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(tokenize_text)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_wskazania)
            df.loc[:, column] = df[column].apply(lemmatize_text)
            df.loc[:, column] = df[column].apply(remove_duplicated_words)

In [47]:
text_cleaned= process_text_columns(text_extracted)

Sample check specific row:

In [48]:
text_extracted.iloc[865:866,:]

,filename,nazwa,sklad,wskazania
865,Charakterystyka-47283-2023-10-31-15084_D-2023-11-10.pdf,ebozan,ebozan torasemidu,pochodzenie tabletka nadciśnienie młodzież wiek rok


Manual search for some files:
- test 
- note: 128 rows with high blood pressure (nadcisnienie)

In [51]:
#searched_file = 'Charakterystyka-36343-20170830000000-3286_A-20170921000413.pdf'
searched_file = ['Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf', 'Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf']
kolumna = 'filename'
#filtered_df = text_extracted[text_extracted[kolumna].str.contains(wyszukiwanie)]
filtered_df = text_extracted[text_extracted[kolumna].str.contains('|'.join(searched_file))]
filtered_df


,filename,nazwa,sklad,wskazania
431,Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf,combogesic,paracetamol,ból głowa plecy zębów mięśni objaw przeziębieć grypa gardło gorączka
602,Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf,enplerasa,eplerenon,eplerenon betaadrenolityka ryzyko umieralność zachorowalność przyczyna stan zaburzenie czynność komora serce ventricular ejection objaw niewydolność zawał klasa nyha punkt


In [53]:
print(f"Found {filtered_df.shape[0]} for searched term in : {searched_file}.")

Found 2 for searched term: ['Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf', 'Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf'].


# utils_data_cleaning.py - changes

testing the functions

In [54]:
import spacy
import re
import pandas as pd
from datetime import datetime

nlp = spacy.load("pl_core_news_lg")


def clean_formatting(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.replace('\n', ' ').strip().lower()  # Normalize text (convert to lowercase)
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'\d+', '', text)  # Remove digits
    return text

def remove_stop_words(text, specified_words_to_remove):
    words = text.split()
    filtered_words = [word for word in words if word not in specified_words_to_remove]
    return ' '.join(filtered_words)

def remove_duplicated_words(text):
    words = text.split()
    unique_words = list(dict.fromkeys(words))  # Maintain order and remove duplicates
    return ' '.join(unique_words)


def lemmatize_text(text):
    doc = nlp(text)  
    lemmatized_text = " ".join(token.lemma_ for token in doc if token.pos_ == "NOUN" or token.pos_ == "PROPN")  
    return lemmatized_text

def process_text_columns(df):
    columns_to_process = ['nazwa', 'sklad', 'wskazania']
    stop_words_nazwa = ['nazwa', 'produktu', 'leczniczego', 'mg']
    stop_words_sklad = ['skład', 'jakościowy', 'ilościowy',  "pełny", "wykaz",  "substancji", \
                         "pomocniczych", "patrz", "punkt", "substancje", "pomocnicze", "mg", "znanym", "każdy", "każda"]
    stop_words_wskazania = [ "wskazania", " do ", "stosowania", "zapobieganie", "stosowania", "wskazany", "wskazanie",\
                            "leczniczy", "leczenie", "leczeniu", " i ", "i ", " e " " o ", " u ", " z ", " przy ", " od ", " do ", " na ", " w ", " oraz ", " kiedy ", " który ", " która ", " które ", " których ",  "pacjent", "patrz", "punkty", "osób", "produkt", "substancja", "pomocniczy", "pomocnicza" , \
                                "metody", "celu", "mieszaną"]
    
    for column in columns_to_process:
        if column == 'nazwa':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_nazwa)
        #    df.loc[:, column] = df[column].apply(lemmatize_text)
            df.loc[:, column] = df[column].apply(remove_duplicated_words)


        elif column == 'sklad':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_sklad)
            df.loc[:, column] = df[column].apply(lemmatize_text)
            df.loc[:, column] = df[column].apply(remove_duplicated_words)
            
        
        elif column == 'wskazania':
            df.loc[:, column] = df[column].apply(clean_formatting)
            df.loc[:, column] = df[column].apply(remove_stop_words, specified_words_to_remove=stop_words_wskazania)
            df.loc[:, column] = df[column].apply(lemmatize_text)
            df.loc[:, column] = df[column].apply(remove_duplicated_words)
    
    return df


In [55]:
text_cleaned= process_text_columns(text_extracted)

In [56]:
type(text_cleaned)

pandas.core.frame.DataFrame

In [57]:
text_cleaned.iloc[695:696,:]

,filename,nazwa,sklad,wskazania
695,Charakterystyka-42003-2023-11-17-16860_B-2024-02-21.pdf,mantreda,rywaroksaban,profilaktyka choroba staw zatorowość punkt


In [58]:
#testing search after column split - 128 rows with high blood pressure -  nadcisnienie

#wyszukiwanie = 'Charakterystyka-36343-20170830000000-3286_A-20170921000413.pdf'
wyszukiwanie = ['Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf', 'Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf']
kolumna = 'filename'
#filtered_df = text_extracted[text_extracted[kolumna].str.contains(wyszukiwanie)]
filtered_clean = text_cleaned[text_cleaned[kolumna].str.contains('|'.join(wyszukiwanie))]
filtered_clean


,filename,nazwa,sklad,wskazania
431,Charakterystyka-33025-2023-11-09-15802_A-2023-11-14.pdf,combogesic,paracetamol,ból głowa plecy ząb mięśni objaw przeziębieć grypa gardło gorączka
602,Charakterystyka-38481-20190517000000-6797_B-20190616000045.pdf,enplerasa,eplerenon,eplerenon betaadrenolityk ryzyko umieralność zachorowalność przyczyna stan zaburzenie czynność komora serce ventricular ejection objaw niewydolność zawał klasa nyha punkt


In [59]:
def convert_to_dict(df):
    list_of_docs = []  
    for index, row in df.iterrows():
        doc = {
            "title": row["nazwa"],  
            "text": row["sklad"] + " " + row["wskazania"],  
            "filename": row["filename"],  
            "timestamp": datetime.now()  
        }
        list_of_docs.append(doc)
    return list_of_docs

In [60]:
list_of_docs = convert_to_dict(text_cleaned)

# utils_search_engine.py

Testing search engine --QUERY (first one)

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def prepare_tfidf_representation(documents):
    corpus = [doc['text'] for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    return vectorizer, tfidf_matrix

In [62]:
# BRANCH 11
def search_product_by_indication(wskazania_pattern, products, vectorizer, tfidf_matrix):
    if not products:
        return []
    lemmatized_wskazania = lemmatize_text(wskazania_pattern)
    query_tfidf = vectorizer.transform([lemmatized_wskazania])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)
    scores = list(enumerate(cosine_similarities[0]))
    rounded_scores = [(index, round(score, 3)) for index, score in scores]
    sorted_rounded_scores = sorted(rounded_scores, key=lambda x: x[1], reverse=True)
    matching_products = [{'product_name': products[idx]['title'], 'filename': products[idx]['filename'], 'score': score} for idx, score in sorted_rounded_scores]
    return matching_products


In [63]:
tfidf_vectorizer, tfidf_matrix = prepare_tfidf_representation(list_of_docs)

In [64]:
q_input = 'cukrzyca'

In [65]:
def capitalize_first_letter(text):
    """
    Capitalize the first letter of the text.
    If the text is empty, return it as is.
    """
    if text:
        return text[0].upper() + text[1:]
    else:
        return text

wskazania_pattern = (q_input) 
matching_products = search_product_by_indication(wskazania_pattern, list_of_docs, tfidf_vectorizer, tfidf_matrix)
threashold = 0.0
filtered_products = [product for product in matching_products if product['score'] > threashold]
for product in filtered_products:
    
    product_name_capitalized = capitalize_first_letter(product["product_name"])
    print(f'{product["filename"]},"{product_name_capitalized}",{product["score"]:.3f}')

Charakterystyka-43642-2022-09-23-12032_B-2022-10-05.pdf,"Telmisartan aurovitas",0.305
Charakterystyka-43643-2022-09-23-12032_B-2022-10-05.pdf,"Telmisartan aurovitas",0.305
Charakterystyka-38066-20210108000000-9169_B-20210123000830.pdf,"Telmisartan orion",0.281
Charakterystyka-46194-2023-06-26-12511_D-2023-07-14.pdf,"Lisinopril grindeks",0.267
Charakterystyka-45664-2023-06-19-14384_B-2023-07-18.pdf,"Dabigatran etexilate pharma",0.258
Charakterystyka-44797-2022-06-30-11620_B-2022-07-08.pdf,"Telmisartan medical valley",0.256
Charakterystyka-44155-2023-11-15-14691_D-2023-12-10.pdf,"Gribero",0.255
Charakterystyka-46619-2023-12-20-15605_D-2024-01-08.pdf,"Danengo",0.255
Charakterystyka-44292-2024-02-19-15518_D-2024-03-10.pdf,"Dabigatran etexilate orion",0.248
Charakterystyka-41664-2022-01-27-1748_D-2022-02-21.pdf,"Lisinopril aurovitas",0.244
Charakterystyka-43822-2023-08-04-15331_B-2023-10-10.pdf,"Ib wasedoc",0.244
Charakterystyka-45217-2023-02-02-2040_D-2023-03-22.pdf,"Mirexan",0.244
Charakt

In [29]:
print(f"Number of outputs found: {len(filtered_products)}")

Number of outputs found: 26


## utils_search_similar.py

1. user provides name of the file or path to it
2. file is read by function read_documents
3. file is split for columns and cleaned
4. file i lematized
5. file is compared with the other indications from list_docs as for ingredients and indication, NOT name
6. product list output is formattes in the same way as the query.

In [66]:
new_folder_path = "/Users/LILI/Projects_studia/_PORTFOLIO/REMOTE/nlp-group2/docs/search_data"
new_file = load_to_pd(new_folder_path)
new_file_extracted = extract_columns(new_file, 'CHPCL') 
new_file_cleaned = process_text_columns(new_file_extracted)

def convert_to_dict_new_file(df):
    list_of_docs = []  
    for index, row in df.iterrows():
        doc = {
            "filename": row["filename"], 
            "nazwa": row["nazwa"],  
            "sklad": row["sklad"],
            "wskazania":  row["wskazania"]
        }
        list_of_docs.append(doc)
    return list_of_docs
new_file_list_of_docs = convert_to_dict_new_file(new_file_extracted)
new_file_sklad = [doc['sklad'] for doc in new_file_list_of_docs]
new_file_wskazania = [doc['wskazania'] for doc in new_file_list_of_docs]

In [67]:
text_df = load_to_pd(input_path)
text_extracted = extract_columns(text_df, 'CHPCL') 
text_cleaned= process_text_columns(text_extracted)
text_dict= convert_to_dict_new_file(text_cleaned)
text_dict_sklad = [doc['sklad'] for doc in text_dict]
text_dict_wskazania = [doc['wskazania'] for doc in text_dict]

In [68]:
new_file_list_of_docs

[{'filename': 'Charakterystyka-173-2023-06-15-13773_N-2023-06-29.pdf',
  'nazwa': 'nicergolin tabletki',
  'sklad': 'tabletka nicergoliny nicergolinum substancja działać produkt laktoza',
  'wskazania': 'otępienie nicergolina dorosły'}]

In [70]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

def run_similarity_model(list_of_docs):

    model = SentenceTransformer('LaBSE')
    doc_embeddings = model.encode([doc['text'] for doc in list_of_docs])

    index = faiss.IndexFlatL2(doc_embeddings[0].shape[0])  # Initialize the index
    index.add(np.array(doc_embeddings)) 

    while True:
        query = input("Enter the file name to list similar products (top 5 outputs): ")
        query_embedding = model.encode([query])[0]
        _, indices = index.search(np.array([query_embedding]), 5)

        top_documents = [(list_of_docs[idx], 1.0) for idx in indices[0]]

        for i, (doc, similarity) in enumerate(top_documents, start=1):
            print(f"Document {i}:")
            print(f"Similarity Score: {similarity}")
            print(f"Title: {doc['title']}")
            print(f"Text: {doc['text']}")
            print(f"Filename: {doc['filename']}")
            print()

# Check versions


In [71]:
import pandas as pd 
pd.__version__

'2.2.1'

In [72]:
import subprocess

packages = [
    "nltk==3.8.1",
    "PyMuPDF==1.24.2",
    "scikit-learn==1.4.1.post1",
    "sentence-transformers==2.5.1",
    "spacy==3.7.4",
    "torch==2.2.1",
    "transformers==4.38.2",
    "faiss-cpu==1.8.0",
    "pandas==2.2.1"
]

for package in packages:
    package_name = package.split('==')[0]
    try:
        result = subprocess.run(["pip", "show", package_name], capture_output=True, text=True)
        if result.returncode == 0:
            version_info = [line for line in result.stdout.split('\n') if "Version" in line]
            if version_info:
                print(f"{package_name} is installed: {version_info[0]}")
            else:
                print(f"{package_name} is installed but version information is not available.")
        else:
            print(f"{package_name} is not installed.")
    except Exception as e:
        print(f"An error occurred while checking {package_name}: {str(e)}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


nltk is installed: Version: 3.8.1
PyMuPDF is installed: Version: 1.24.2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


scikit-learn is installed: Version: 1.4.1.post1
sentence-transformers is installed: Version: 2.5.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

transformers is installed: Version: 4.38.2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


faiss-cpu is installed: Version: 1.8.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pandas is installed: Version: 2.2.1
